In [101]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking, Conv1D, Flatten, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
import numpy as np
import pandas as pd
import yfinance as yf

In [3]:
from pathlib import Path
DATA_PATH = Path("raw_data/patterns_dbs")
DATA_PATH.exists()

True

In [4]:
data = {filepath.stem: pd.read_csv(filepath) for filepath in DATA_PATH.glob("*.csv")}

In [132]:
data["PZU_96_desc._broad._wedge"]

,Date,Open,High,Low,Close,Adjusted_close,Volume
0,2023-06-27,40.35,40.67,39.30,39.77,39.77,954983
1,2023-06-28,39.77,40.21,39.41,39.73,39.73,766967
2,2023-06-29,40.00,40.24,39.70,39.93,39.93,1102269
3,2023-06-30,40.29,40.52,39.32,39.41,39.41,1295284
4,2023-07-03,39.41,39.91,39.41,39.73,39.73,1358342
5,2023-07-04,39.80,39.80,39.04,39.04,39.04,490231
6,2023-07-05,39.08,39.08,38.00,38.36,38.36,1528430
7,2023-07-06,38.10,38.40,37.55,38.04,38.04,2498022
8,2023-07-07,38.18,38.62,38.08,38.51,38.51,976168
9,2023-07-10,38.50,38.63,38.04,38.24,38.24,677229


In [6]:
types1 = [type(k) for k in data.keys()]
types1

[str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str,
 str

In [7]:
data["PZU_96_desc._broad._wedge"]

,Date,Open,High,Low,Close,Adjusted_close,Volume
0,2023-06-27,40.35,40.67,39.30,39.77,39.77,954983
1,2023-06-28,39.77,40.21,39.41,39.73,39.73,766967
2,2023-06-29,40.00,40.24,39.70,39.93,39.93,1102269
3,2023-06-30,40.29,40.52,39.32,39.41,39.41,1295284
4,2023-07-03,39.41,39.91,39.41,39.73,39.73,1358342
5,2023-07-04,39.80,39.80,39.04,39.04,39.04,490231
6,2023-07-05,39.08,39.08,38.00,38.36,38.36,1528430
7,2023-07-06,38.10,38.40,37.55,38.04,38.04,2498022
8,2023-07-07,38.18,38.62,38.08,38.51,38.51,976168
9,2023-07-10,38.50,38.63,38.04,38.24,38.24,677229


In [8]:
len(data)

359

In [9]:
tensor_list = []
label_list = []

for key, df in data.items():
    tensor_list.append(df[['Open', 'High', 'Low', 'Close']].values)
    label_list.append(np.random.randint(1, high=df.index.values.max(), size=2))


In [10]:
np.random.randint(1, high=df.index.values.max(), size=2)

array([22, 15])

In [11]:
tensor_list

[array([[40.35, 40.67, 39.3 , 39.77],
        [39.77, 40.21, 39.41, 39.73],
        [40.  , 40.24, 39.7 , 39.93],
        [40.29, 40.52, 39.32, 39.41],
        [39.41, 39.91, 39.41, 39.73],
        [39.8 , 39.8 , 39.04, 39.04],
        [39.08, 39.08, 38.  , 38.36],
        [38.1 , 38.4 , 37.55, 38.04],
        [38.18, 38.62, 38.08, 38.51],
        [38.5 , 38.63, 38.04, 38.24],
        [38.5 , 38.74, 37.86, 38.01],
        [38.17, 39.11, 38.17, 39.06],
        [39.1 , 39.25, 38.6 , 38.94],
        [38.92, 39.12, 38.45, 38.6 ]]),
 array([[375.3, 378.2, 368.2, 369.2],
        [370. , 370.2, 357. , 359.4],
        [362.9, 367.3, 354.2, 360. ],
        [360. , 363.5, 351.4, 352.8],
        [352.2, 365.8, 351.1, 365.3],
        [364.9, 367.6, 360.2, 365. ],
        [364.6, 369.7, 361.4, 365.2],
        [367.2, 373.7, 363.3, 366.1],
        [366.7, 371. , 363. , 363.5],
        [366.9, 369.9, 358. , 368.6],
        [368. , 370.6, 361.7, 363. ],
        [363. , 370. , 361.5, 361.9],
        [3

In [12]:
data_pad = pad_sequences(tensor_list, dtype='float32', padding='post', value=-1)
data_pad[0]

array([[40.35, 40.67, 39.3 , 39.77],
       [39.77, 40.21, 39.41, 39.73],
       [40.  , 40.24, 39.7 , 39.93],
       [40.29, 40.52, 39.32, 39.41],
       [39.41, 39.91, 39.41, 39.73],
       [39.8 , 39.8 , 39.04, 39.04],
       [39.08, 39.08, 38.  , 38.36],
       [38.1 , 38.4 , 37.55, 38.04],
       [38.18, 38.62, 38.08, 38.51],
       [38.5 , 38.63, 38.04, 38.24],
       [38.5 , 38.74, 37.86, 38.01],
       [38.17, 39.11, 38.17, 39.06],
       [39.1 , 39.25, 38.6 , 38.94],
       [38.92, 39.12, 38.45, 38.6 ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
       [-1.  , -1.  , -1.  , -1.  ],
 

In [13]:
data_pad.shape

(359, 82, 4)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_pad, label_list, test_size=0.30)

In [72]:
y_train

array([[25,  6],
       [ 7,  8],
       [10, 12],
       [ 7, 10],
       [ 2,  4],
       [15,  4],
       [12,  5],
       [16,  2],
       [46, 44],
       [47, 27],
       [ 1,  2],
       [ 7,  4],
       [ 7,  7],
       [ 3,  2],
       [ 4,  2],
       [29, 29],
       [11, 32],
       [ 1,  3],
       [ 3,  1],
       [ 3,  8],
       [ 3,  1],
       [17, 13],
       [ 7,  8],
       [ 3,  3],
       [ 8,  5],
       [ 6, 18],
       [ 1,  3],
       [10, 12],
       [ 2,  1],
       [ 2,  4],
       [ 8, 21],
       [ 5,  8],
       [11,  7],
       [28, 18],
       [37, 19],
       [ 9,  2],
       [ 7,  3],
       [ 1,  3],
       [ 1,  9],
       [ 4, 16],
       [ 9, 13],
       [32,  8],
       [ 1,  4],
       [13,  4],
       [52,  5],
       [ 2,  3],
       [13,  8],
       [19,  4],
       [ 4,  2],
       [ 4,  4],
       [ 4,  4],
       [37,  5],
       [ 5, 19],
       [16, 15],
       [ 1, 12],
       [11,  2],
       [32, 41],
       [30, 58],
       [ 2,  4

In [16]:
print("X_train shape:", X_train.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)
print("y_train shape:", y_train.shape)

X_train shape: (251, 82, 4)
y_train shape: (251, 2)


In [37]:
def initialize_model_LSTM():
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=(82,4)))
    model.add(LSTM(units=32, activation='relu'))

    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=2, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model


In [119]:
def initialize_model_CNN():
    model = Sequential()

    model.add(Masking(mask_value=-1, input_shape=(82,4)))
    model.add(Conv1D(32, activation='relu', kernel_size=3, kernel_regularizer=regularizers.L1L2(l1=1e-3, l2=1e-3))),
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(32, activation='relu', kernel_size=3))
    
    model.add(Flatten())
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=16, activation='relu'))
    model.add(Dense(units=2, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return model

In [56]:
from tensorflow import keras
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 5, restore_best_weights=True)

In [57]:
model = initialize_model_LSTM()

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
7/7 [==============================] - 3s 76ms/step - loss: 2451970195456.0000 - mae: 118675.0312 - val_loss: 6856529920.0000 - val_mae: 17490.5176
Epoch 2/50
7/7 [==============================] - 0s 46ms/step - loss: 20664463360.0000 - mae: 13897.0342 - val_loss: 664969920.0000 - val_mae: 6471.7817
Epoch 3/50
7/7 [==============================] - 0s 50ms/step - loss: 216112627712.0000 - mae: 35526.4844 - val_loss: 1750386304.0000 - val_mae: 9030.2148
Epoch 4/50
7/7 [==============================] - 0s 46ms/step - loss: 176023945216.0000 - mae: 28781.2383 - val_loss: 17336724.0000 - val_mae: 923.0117
Epoch 5/50
7/7 [==============================] - 0s 48ms/step - loss: 328827424.0000 - mae: 2278.7600 - val_loss: 24575124.0000 - val_mae: 1105.3521
Epoch 6/50
7/7 [==============================] - 0s 56ms/step - loss: 108549384.0000 - mae: 1471.9376 - val_loss: 16278212.0000 - val_mae: 825.6120
Epoch 7/50
7/7 [==============================] - 0s 52ms/step - loss: 76001016

In [58]:
res = model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 10ms/step - loss: 9796966.0000 - mae: 791.5910


In [130]:
model = initialize_model_CNN()

model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    shuffle = True,
    batch_size=32,
    epochs = 50,
    callbacks = [es],
    verbose = 1
)

Epoch 1/50
7/7 [==============================] - 1s 33ms/step - loss: 168663.7031 - mae: 66.3524 - val_loss: 25327.4160 - val_mae: 39.8681
Epoch 2/50
7/7 [==============================] - 0s 12ms/step - loss: 129265.0000 - mae: 49.5047 - val_loss: 15889.1582 - val_mae: 36.3866
Epoch 3/50
7/7 [==============================] - 0s 11ms/step - loss: 16561.2695 - mae: 31.5149 - val_loss: 6740.6104 - val_mae: 27.5154
Epoch 4/50
7/7 [==============================] - 0s 11ms/step - loss: 10404.2070 - mae: 26.8918 - val_loss: 1995.6033 - val_mae: 20.4697
Epoch 5/50
7/7 [==============================] - 0s 11ms/step - loss: 5801.7437 - mae: 22.5314 - val_loss: 1079.3160 - val_mae: 17.1883
Epoch 6/50
7/7 [==============================] - 0s 11ms/step - loss: 2171.1042 - mae: 18.2450 - val_loss: 894.6569 - val_mae: 16.6247
Epoch 7/50
7/7 [==============================] - 0s 11ms/step - loss: 817.7632 - mae: 15.4723 - val_loss: 1052.2986 - val_mae: 17.1603
Epoch 8/50
7/7 [===================

In [131]:
res = model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 3ms/step - loss: 375.5141 - mae: 11.9418


In [63]:
pred = model.predict(X_test)
pred[0], y_test[0]

4/4 [==============================] - 0s 4ms/step


(array([3.0630207, 1.355848 ], dtype=float32), array([18, 11]))